compute the powerspectrum multipoles for halo catalog with $\sum m_\nu = 0. \mathrm{eV}$ at $z = 0$

In [6]:
import numpy as np
import nbodykit.lab as NBlab

In [2]:
import env
import data as Data 
import forwardmodel as FModel

In [3]:
import matplotlib as mpl 
import matplotlib.pyplot as pl 
mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['axes.xmargin'] = 1
mpl.rcParams['xtick.labelsize'] = 'x-large'
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['xtick.major.width'] = 1.5
mpl.rcParams['ytick.labelsize'] = 'x-large'
mpl.rcParams['ytick.major.size'] = 5
mpl.rcParams['ytick.major.width'] = 1.5
mpl.rcParams['legend.frameon'] = False
%matplotlib inline

read in the halo catalog 

In [4]:
halos = Data.NeutHalos(0.0, 1, 4)

Compute real-space $P(k)$

In [9]:
halos.columns

['Mass', 'Position', 'Radius', 'Selection', 'Value', 'Velocity', 'Weight']

In [11]:
print halos['Radius'].shape

(0,)


In [7]:
pk_halo_real = NBlab.FFTPower(halos, mode='1d', Nmesh=256)

ValueError: trying to paint particle source to mesh, but no particles were found!